In [1]:
!pip install mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 25.0 MB/s eta 0:00

In [2]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import seaborn as sn

import random

In [3]:
from google.colab import  files
uploaded = files.upload()

Saving mapa.txt to mapa.txt


In [4]:
class FiremanAgent(Agent):
    def __init__(self, unique_id, model, point=None):
      super().__init__(unique_id, model)
      self.point = point
      self.actionPoints = 4
      self.carryState = 1

    def step(self):
      myTurn = True
      if self.point is not None:
        path = self.dijkstra(self.pos, self.model.interestPoints[self.point].pos)[0]
        while(myTurn):
          if path and self.actionPoints >= self.calculateSteps(self.pos, path[0]):
            self.actionPoints -= self.calculateSteps(self.pos, path[0])
            self.model.grid.move_agent(self, path[0])
            path.pop(0)
          elif path and self.actionPoints < self.calculateSteps(self.pos, path[0]):
            if self.actionPoints + 4 > 8:
              self.actionPoints = 8
              myTurn = False
            else:
              self.actionPoints += 4
              myTurn = False
          if self.pos == self.model.interestPoints[self.point].pos:
            if self.model.interestPoints[self.point].alert == 2:
              self.model.interestPoints[self.point].alert = 0
              self.carryState = 2
              closestExit = None
              minDistance = 100
              for exit in self.model.outSide:
                distance = self.dijkstra(self.pos, exit.pos)[1]
                if distance < minDistance:
                  minDistance = distance
                  closestExit = exit
              self.model.cells[self.pos[0]][self.pos[1]].alert = 0
              self.model.interestPoints.append(closestExit)
              self.model.interestPoints[self.point] = self.model.generateNewInterestPoint()
              self.point = len(self.model.interestPoints) - 1
            elif self.model.cells[self.pos[0]][self.pos[1]] in self.model.outSide and self.carryState == 2:
              print("Vida salvada")
              self.carryState = 1
              self.model.savedLifes += 1
              self.model.interestPoints.remove(self.model.interestPoints[self.point])
            elif self.model.interestPoints[self.point].alert == 1:
              print("Falsa alarma")
              self.model.cells[self.pos[0]][self.pos[1]].alert = 0
              self.model.interestPoints[self.point] = self.model.generateNewInterestPoint()
            myTurn = False
      else:
        closest_fire_point = None
        min_distance = 100
        for fire_point in self.model.firePoints:
          distance = self.dijkstra(self.pos, fire_point.pos)[1]
          if distance < min_distance:
            min_distance = distance
            closest_fire_point = fire_point
        print(f"Agente {self.unique_id} asignado al fuego en {closest_fire_point.pos}")
        path = self.dijkstra(self.pos, closest_fire_point.pos)[0]
        while(myTurn):
          if path and self.actionPoints >= self.calculateSteps(self.pos, path[0]):
            self.actionPoints -= self.calculateSteps(self.pos, path[0])
            self.model.grid.move_agent(self, path[0])
            path.pop(0)
          elif path and self.actionPoints < self.calculateSteps(self.pos, path[0]):
            if self.actionPoints + 4 > 8:
              self.actionPoints = 8
              myTurn = False
            else:
              self.actionPoints += 4
              myTurn = False
          else:
            print("No hay camino al fuego")
            myTurn = False
      if self.carryState == 1:
        self.point = None


    def dijkstra(self, start, end):
      dijkstraMap = {}
      path = []
      for x in range(self.model.grid.height):
        for y in range(self.model.grid.width):
          dijkstraMap[(y, x)] = {"previousCell": None, "steps": None}
      if start in dijkstraMap:
        dijkstraMap[start]["steps"] = 0
        dijkstraMap[start]["previousCell"] = start
        queue = [start]
        while len(queue) > 0:
          for neighbor in self.model.grid.get_neighborhood(queue[0], moore=False):
            if 0 <= neighbor[0] < self.model.grid.width and 0 <= neighbor[1] < self.model.grid.height:
              if dijkstraMap[neighbor]["steps"] is None:
                dijkstraMap[neighbor]["steps"] = self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"]
                dijkstraMap[neighbor]["previousCell"] = queue[0]
                queue.append(neighbor)
              elif (dijkstraMap[neighbor]["steps"] > self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"]):
                dijkstraMap[neighbor]["steps"] = (self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"])
                dijkstraMap[neighbor]["previousCell"] = queue[0]
                queue.append(neighbor)
          queue.pop(0)
        cell = end
        while cell != start:
          path.insert(0, cell)
          cell = dijkstraMap[cell]["previousCell"]
        return path, dijkstraMap[end]["steps"]
      else:
        print("No existe esa celda de inicio")
        return path, dijkstraMap[end]["steps"]

    def calculateSteps(self, start, end):
          actionPointsCost = 0
          # print(start, end)
          if 0 <= end[0] < len(self.model.cells) and 0 <= end[1] < len(self.model.cells[0]):
            if start[0] < end[0]:
              if self.model.cells[end[0]][end[1]].up or self.model.cells[start[0]][start[1]].down:
                actionPointsCost += 4
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
            elif start[0] > end[0]:
              if self.model.cells[end[0]][end[1]].down or self.model.cells[start[0]][start[1]].up:
                actionPointsCost += 4
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
            elif start[1] < end[1]:
              if self.model.cells[end[0]][end[1]].left or self.model.cells[start[0]][start[1]].right:
                actionPointsCost += 4
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
            elif start[1] > end[1]:
              if self.model.cells[end[0]][end[1]].right or self.model.cells[start[0]][start[1]].left:
                actionPointsCost += 4
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
          actionPointsCost = actionPointsCost + 1 * self.carryState
          return actionPointsCost


In [5]:
class Cell():
  def __init__(self, x, y, wall):
    self.pos = (x, y)
    self.wallHealth = [0, 0, 0, 0]

    if wall[0] == '1':
        self.up = True
        self.wallHealth[0] = 2
    else: self.up = False
    if wall[1] == '1':
        self.left = True
        self.wallHealth[1] = 2
    else: self.left = False
    if wall[2] == '1':
        self.down = True
        self.wallHealth[2] = 2
    else: self.down = False
    if wall[3] == '1':
        self.right = True
        self.wallHealth[3] = 2
    else: self.right = False

    self.alert = 0 # 1- false alarm    2 - victim
    self.fire = 0 # 1 - smoke    2 - fire
    self.door = [] # pos of the connected cell with a door & if is open    [] if no door
    self.entrance = False # True if the current cell is a access point to the structure


In [70]:
with open('mapa.txt', 'r') as map:
    text = map.read()

walls = []
for i in range(8):
    for j in range(6):
        new_wall = text[:4]
        walls.append(new_wall)
        text = text[5:]

alerts = []
for i in range(3):
    pos_alert_x = text[0]
    pos_alert_y = text[2]
    pos_alert_state = text[4]
    text = text[6:]
    alerts.append( (pos_alert_x, pos_alert_y, pos_alert_state) )

fires = []
for i in range(10):
    pos_fire_x = text[0]
    pos_fire_y = text[2]
    text = text[4:]
    fires.append( (pos_fire_x, pos_fire_y) )

doors = []
for i in range(8):
    pos_doorA_x = text[0]
    pos_doorA_y = text[2]
    pos_doorB_x = text[4]
    pos_doorB_y = text[6]
    text = text[8:]
    doors.append( ( (pos_doorA_x, pos_doorA_y), (pos_doorB_x, pos_doorB_y) ) )

exits = []
for i in range(4):
    pos_exit_x = text[0]
    pos_exit_y = text[2]
    text = text[4:]
    exits.append( (pos_exit_x, pos_exit_y) )

cells = []
for i in range(6):
    for j in range(8):
        w = walls[0]
        del walls[0]

        c = Cell(i + 1,j + 1,w)
        cells.append(c)

        if (str(i + 1), str(j + 1), 'v') in alerts:
            c.alert = 2
        elif (str(i + 1), str(j + 1), 'f') in alerts:
            c.alert = 1

        if (str(i + 1), str(j + 1)) in fires:
            c.fire = 2

        for d in doors:
            if (str(i + 1), str(j + 1)) == d[0]:
                c.door = d[1]
            elif (str(i + 1), str(j + 1)) == d[1]:
                c.door = d[0]

        if (str(i + 1), str(j + 1)) in exits:
            c.entrance = True

        #print(f"{c.pos}: {c.up} - {c.left} - {c.down} - {c.right}   A: {c.alert}   F: {c.fire}   D: {c.door}    E: {c.entrance}")

new_cells = [
    Cell(0, 0, "0000"),
    Cell(0, 1, "0010"),
    Cell(0, 2, "0010"),
    Cell(0, 3, "0010"),
    Cell(0, 4, "0010"),
    Cell(0, 5, "0010"),
    Cell(0, 6, "0010"),
    Cell(0, 7, "0010"),
    Cell(0, 8, "0010"),
    Cell(0, 9, "0000"),
]

outSide = new_cells
cells = new_cells + cells

for i in range (1, 7):
    c = Cell(i, 0, "0001")
    cells.insert(i * 10, c)
    outSide.append(c)

    c = Cell(i, 9, "0100")
    cells.insert((i * 10) + 9, c)
    outSide.append(c)

new_cells = [
    Cell(7, 0, "0000"),
    Cell(7, 1, "1000"),
    Cell(7, 2, "1000"),
    Cell(7, 3, "1000"),
    Cell(7, 4, "1000"),
    Cell(7, 5, "1000"),
    Cell(7, 6, "1000"),
    Cell(7, 7, "1000"),
    Cell(7, 8, "1000"),
    Cell(7, 9, "0000"),
]

outSide = outSide + new_cells
cells = cells + new_cells

map = [[None for _ in range(10)] for _ in range(8)]
for cell in cells:
    y, x = cell.pos
    map[y][x] = cell
    #print(f"{map[y][x].pos}: {map[y][x].up} - {map[y][x].left} - {map[y][x].down} - {map[y][x].right}   A: {map[y][x].alert}   F: {map[y][x].fire}   D: {map[y][x].door}    E: {map[y][x].entrance}")


In [72]:
class MapModel(Model):
    def __init__(self, num_agents, cells, outSide):
        super().__init__()
        self.savedLifes = 0
        self.outSide = outSide
        self.cells = cells
        self.running = True
        self.height = 8
        self.width = 10
        self.structural_Damage_Left = 24
        self.num_agents = num_agents

        self.grid = MultiGrid(self.height, self.width, False)
        self.schedule = SimultaneousActivation(self)

        self.interestPoints = [cell for row in cells for cell in row if cell.alert != 0]
        self.firePoints = [cell for row in cells for cell in row if cell.fire == 2]
        self.smokePoints = []

        # empty_positions = [(x, y) for x in range(self.height) for y in range(self.width)]

        # for i in range(self.num_agents):
        #     random_pos = self.random.choice(empty_positions)
        #     a = FiremanAgent(i, self)  # No asignamos punto inicialmente
        #     empty_positions.remove(random_pos)
        #     self.schedule.add(a)
        #     self.grid.place_agent(a, random_pos)
        #     print(f"Agente {a.unique_id} inicio en posición {a.pos}")

    def step(self):
        # minSteps = 100

        # for i in range(len(self.interestPoints)):
        #     print("Punto de interes", self.interestPoints[i].pos)
        #     closestAgent = None

        #     for agent in self.schedule.agents:
        #         if agent.point is None:
        #             steps = agent.dijkstra(agent.pos, self.interestPoints[i].pos)[1]
        #             if steps < minSteps:
        #                 minSteps = steps
        #                 closestAgent = agent.unique_id

        #     for agent in self.schedule.agents:
        #         if closestAgent == agent.unique_id:
        #             agent.point = i
        #             print(f"Agente {agent.unique_id} asignado al punto de interés {i}")
        #             minSteps = 100
        #             break

        #self.schedule.step()

        for fire in self.firePoints:
            print(f"{fire.pos} {fire.fire}")

        for smoke in self.smokePoints:
            print(f"{smoke.pos} {smoke.fire}")

        self.spreadFire()

    def getNeighbors(self, x, y):
        neighbors = []

        if x > 1:
            neighbors.append((x - 1, y))
        if x < self.height:
            neighbors.append((x + 1, y))
        if y > 1:
            neighbors.append((x, y - 1))
        if y < self.width:
            neighbors.append((x, y + 1))

        return neighbors

    def spreadFire(self):
        def smokeIntoFire(x, y, neighbors):
            is_smoke = False

            for neighbor in neighbors:
                for s in self.smokePoints:
                    if neighbor == s.pos:
                        is_smoke = True
                        break

            if not is_smoke: return

            for neighbor in neighbors:
                for s in self.smokePoints:
                    if neighbor == s.pos:
                        s.fire +=1
                        self.firePoints.append(s)
                        self.smokePoints.remove(s)
                        print(f"new fire in ({neighbor[0]} {neighbor[1]})")
                        smokeIntoFire(neighbor[0], neighbor[1], self.getNeighbors(neighbor[0], neighbor[1]))

        fireX = random.choice( range(1, self.height - 1) )
        fireY = random.choice( range(1, self.width - 1) )

        neighbors = self.getNeighbors(fireX, fireY)

        for row in self.cells:
            for c in row:
                if (fireX, fireY) == c.pos and c.fire < 2:
                    c.fire += 1

                    if c.fire == 2:
                        self.firePoints.append(c)
                        if c in self.smokePoints:
                            self.smokePoints.remove(c)
                        print(f"new fire in ({fireX} {fireY})")
                        smokeIntoFire(fireX, fireY, neighbors)

                    else:
                        self.smokePoints.append(c)
                        print(f"new smoke in ({fireX} {fireY})")

                        for neighbor in neighbors:
                            for f in self.firePoints:
                                if neighbor == f.pos:
                                    c.fire += 1
                                    self.firePoints.append(c)
                                    if c in self.smokePoints:
                                        self.smokePoints.remove(c)
                                    print(f"new fire in ({fireX} {fireY})")
                                    smokeIntoFire(fireX, fireY, neighbors)
                                    break
                            if neighbor == f.pos: break


                    break
                elif (fireX, fireY) == c.pos:
                    self.explotion(fireX, fireY)
                    break

    def explotion(self, x, y):
        print(f"new explotion in ({x} {y})")

        neighbors = self.getNeighbors(x, y)
        cellNeighbors = []

        for row in self.cells:
            for c in row:
                if c.pos == (x, y):
                    explotedCell = c
                    continue

                for neighbor in neighbors:
                    if neighbor == c.pos:
                        cellNeighbors.append(c)
                        break

        for cn in cellNeighbors:
            print(cn.pos, end="")
        print()

        explotedDoor = []
        explotedNDoor = []

        if len(explotedCell.door) > 0:
            explotedDoor = [a - b for a, b in zip([explotedCell.pos[0], explotedCell.pos[1]], [int(explotedCell.door[0]), int(explotedCell.door[1])] )]
            explotedCell.door = []

            neighborCell = None

            for cn in cellNeighbors:
                if len(cn.door) > 0:
                    explotedNDoor = [a - b for a, b in zip([cn.pos[0], cn.pos[1]], [int(cn.door[0]), int(cn.door[1])] )]
                    cn.door = []
                    neighborCellDoor = cn
                    break

        match explotedDoor:
            case [0, 1]:
                explotedCell.left = False
                print("left door destroyed")
            case [0, -1]:
                explotedCell.right = False
                print("right door destroyed")
            case [1, 0]:
                explotedCell.up = False
                print("top door destroyed")
            case [-1, 0]:
                explotedCell.down = False
                print("bottom door destroyed")

        match explotedNDoor:
            case [0, 1]:
                neighborCellDoor.left = False
                print("neighbor left door destroyed")
            case [0, -1]:
                neighborCellDoor.right = False
                print("neighbor right door destroyed")
            case [1, 0]:
                neighborCellDoor.up = False
                print("neighbor top door destroyed")
            case [-1, 0]:
                neighborCellDoor.down = False
                print("neighbor bottom door destroyed")

        if explotedCell.up:
            self.structural_Damage_Left -= 1
            explotedCell.wallHealth[0] -= 1
            print("top wall damaged")

            if explotedCell.wallHealth[0] == 0:
                print("wall destroyed")
                explotedCell.up = False

        if explotedCell.left:
            self.structural_Damage_Left -= 1
            explotedCell.wallHealth[1] -= 1
            print("left wall damaged")

            if explotedCell.wallHealth[1] == 0:
                print("wall destroyed")
                explotedCell.left = False

        if explotedCell.down:
            self.structural_Damage_Left -= 1
            explotedCell.wallHealth[2] -= 1
            print("bottom wall damaged")

            if explotedCell.wallHealth[2] == 0:
                print("wall destroyed")
                explotedCell.down = False

        if explotedCell.right:
            self.structural_Damage_Left -= 1
            explotedCell.wallHealth[3] -= 1
            print("right wall damaged")

            if explotedCell.wallHealth[3] == 0:
                print("wall destroyed")
                explotedCell.right = False

    def generateNewInterestPoint(self):
        newInterestPointPosition = [(x, y) for x in range(1, self.height - 1) for y in range(1, self.width - 1)]
        random_pos = self.random.choice(newInterestPointPosition)
        self.cells[random_pos[0]][random_pos[1]].alert = self.random.randint(1, 2)
        return self.cells[random_pos[0]][random_pos[1]]


In [73]:
model = MapModel(2, map, outSide)

for i in range(25):

    # for agent in model.schedule.agents:
    #     print()
    #     print(f"Agente {agent.unique_id} en posición {agent.pos}")

    model.step()


(2, 2) 2
(2, 3) 2
(3, 2) 2
(3, 3) 2
(3, 4) 2
(3, 5) 2
(4, 4) 2
(5, 6) 2
(5, 7) 2
(6, 6) 2
new smoke in (5 3)
(2, 2) 2
(2, 3) 2
(3, 2) 2
(3, 3) 2
(3, 4) 2
(3, 5) 2
(4, 4) 2
(5, 6) 2
(5, 7) 2
(6, 6) 2
(5, 3) 1
new explotion in (5 6)
(4, 6)(5, 5)(5, 7)(6, 6)
top wall damaged
left wall damaged
(2, 2) 2
(2, 3) 2
(3, 2) 2
(3, 3) 2
(3, 4) 2
(3, 5) 2
(4, 4) 2
(5, 6) 2
(5, 7) 2
(6, 6) 2
(5, 3) 1
new smoke in (4 6)
new fire in (4 6)
(2, 2) 2
(2, 3) 2
(3, 2) 2
(3, 3) 2
(3, 4) 2
(3, 5) 2
(4, 4) 2
(5, 6) 2
(5, 7) 2
(6, 6) 2
(4, 6) 2
(5, 3) 1
new smoke in (5 8)
new fire in (5 8)
(2, 2) 2
(2, 3) 2
(3, 2) 2
(3, 3) 2
(3, 4) 2
(3, 5) 2
(4, 4) 2
(5, 6) 2
(5, 7) 2
(6, 6) 2
(4, 6) 2
(5, 8) 2
(5, 3) 1
new smoke in (4 3)
new fire in (4 3)
new fire in (5 3)
(2, 2) 2
(2, 3) 2
(3, 2) 2
(3, 3) 2
(3, 4) 2
(3, 5) 2
(4, 4) 2
(5, 6) 2
(5, 7) 2
(6, 6) 2
(4, 6) 2
(5, 8) 2
(4, 3) 2
(5, 3) 2
new explotion in (5 7)
(4, 7)(5, 6)(5, 8)(6, 7)
top wall damaged
right wall damaged
(2, 2) 2
(2, 3) 2
(3, 2) 2
(3, 3) 2
(3, 4) 2
(